In [38]:
import pandas as pd
import numpy as np
from common_utilities import Portfolio, global_path


def main():
    # read the csv file
    df_TradeHistory = pd.read_csv(global_path.tradehistory_silver_file_path)
    # Convert 'datetime' to datetime type
    df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])
    # sort the dataframe by date
    df_TradeHistory = df_TradeHistory.sort_values(by="datetime")
    # replace scrip code to compnay name
    df_Symbol = pd.read_csv(global_path.symbol_silver_file_path)
    df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)
    # Merge df_TradeHistory with df_Symbol on the matching columns
    df_TradeHistory = df_TradeHistory.merge(
        df_Symbol[["scrip_code", "symbol"]],
        left_on="scrip_code",
        right_on="scrip_code",
        how="left",
    )
    # Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
    df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(df_TradeHistory["stock_name"])
    df_TradeHistory = df_TradeHistory[df_TradeHistory["stock_name"] == "NIFTY-CE-24400-11Jul2024"].sort_values(by="datetime")

    portfolio = Portfolio()

    # Apply the function of trade logic to each row of the DataFrame
    data = [portfolio.trade(row.to_dict()) for _, row in df_TradeHistory.iterrows()]
    df_TradeHistory = pd.DataFrame(data)

    # Select columns that end with 'price' or 'amount'
    columns_to_round = [col for col in df_TradeHistory.columns if col.endswith("price") or col.endswith("amount")]
    # Round the values in the selected columns to two decimal places
    df_TradeHistory[columns_to_round] = df_TradeHistory[columns_to_round].round(2)

    return df_TradeHistory

In [39]:
class Portfolio:
    def __init__(self):
        self.stocks = dict()

    def trade(self, record: dict = None):
        stock_name = str(record.get("stock_name"))

        if stock_name not in self.stocks:
            self.stocks[stock_name] = Stock(stock_name)

        record.update(
            self.stocks[stock_name].trade(
                side=str(record.get("side")).upper(),
                price=float(record.get("price")),
                quantity=int(record.get("quantity")),
            )
        )
        return record


class Stock:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        self.avg_price = 0
        self.holding_quantity = 0
        self.holding_amount = 0

    def trade(
        self,
        side: str,
        price: float,
        quantity: int,
    ):
        if self.avg_price == 0:
            pnl_amount = (price - self.avg_price) * quantity
        else:
            pnl_amount = 0

        if side == "BUY":
            pass
        elif side == "SELL":
            quantity = -quantity
        else:
            raise Exception(f"{side} was never excepected")

        amount = quantity * price
        self.holding_quantity += quantity

        if self.holding_quantity == 0:
            self.holding_amount = 0
            self.avg_price = 0
        else:
            if pnl_amount == 0:
                self.holding_amount += amount
            else:
                self.holding_amount += self.avg_price * quantity

            self.avg_price = self.holding_amount / self.holding_quantity

        return {
            "quantity": quantity,
            "amount": amount,
            # "buy_price": buy_price,
            # "buy_amount": buy_amount,
            # "sell_price": sell_price,
            # "sell_amount": sell_amount,
            "avg_price": self.avg_price,
            "holding_quantity": self.holding_quantity,
            "holding_amount": self.holding_amount,
            "pnl_amount": pnl_amount,
        }


main()[["datetime", "side", "quantity", "price", "amount", "holding_quantity", "avg_price", "holding_amount", "pnl_amount"]]

,datetime,side,quantity,price,amount,holding_quantity,holding_price,holding_amount,pnl_amount
0,2024-07-09 12:44:04,BUY,25,89.0,2225.0,25,0.0,0.0,2225.0
1,2024-07-09 12:44:04,BUY,125,89.0,11125.0,150,0.0,0.0,11125.0
2,2024-07-09 13:17:26,SELL,-50,117.1,-5855.0,100,0.0,0.0,5855.0
3,2024-07-09 13:17:26,SELL,-100,117.1,-11710.0,0,0.0,0.0,11710.0
4,2024-07-11 11:05:32,SELL,-25,4.3,-107.5,-25,-0.0,0.0,107.5
5,2024-07-11 14:09:06,BUY,10,2.5,25.0,-15,-0.0,0.0,25.0
6,2024-07-11 14:09:06,BUY,15,2.5,37.5,0,0.0,0.0,37.5


In [40]:
df[["datetime", "side", "quantity", "price", "amount", "holding_quantity", "avg_price", "holding_amount", "pnl_amount"]]

,datetime,side,quantity,price,amount,holding_quantity,holding_price,holding_amount,pnl_amount
0,2024-07-09 12:44:04,BUY,25,89.0,2225.0,25,89.000000,2225.0,0.000000
1,2024-07-09 12:44:04,BUY,125,89.0,11125.0,150,74.166667,11125.0,-1854.166667
2,2024-07-09 13:17:26,SELL,-50,117.1,-5855.0,100,-58.550000,-5855.0,8782.500000
3,2024-07-09 13:17:26,SELL,-100,117.1,-11710.0,0,-inf,-11710.0,inf
4,2024-07-11 11:05:32,SELL,-25,4.3,-107.5,-25,4.300000,-107.5,-0.000000
5,2024-07-11 14:09:06,BUY,10,2.5,25.0,-15,-1.666667,25.0,-41.666667
6,2024-07-11 14:09:06,BUY,15,2.5,37.5,0,inf,37.5,inf


In [55]:
data = {
    "datetime": [
        "2024-07-09 12:44:04",
        "2024-07-09 12:44:04",
        "2024-07-09 13:17:26",
        "2024-07-09 13:17:26",
        "2024-07-11 11:05:32",
        "2024-07-11 14:09:06",
        "2024-07-11 14:09:06",
    ],
    "side": ["BUY", "BUY", "SELL", "SELL", "SELL", "BUY", "BUY"],
    "quantity": [25, 125, 50, 100, 25, 10, 15],
    "price": [89.0, 89.0, 117.1, 117.1, 4.3, 2.5, 2.5],
    "amount": [2225.0, 11125.0, 5855.0, 11710.0, 107.5, 25.0, 37.5],
    "avg_price": [89.0, 89.0, 89.0, 0.0, 4.3, 5.5, 0.0],
    "holding_quantity": [25, 150, 100, 0, -25, -15, 0],
    "holding_amount": [2225.0, 13350.0, 8900.0, 0.0, -107.5, -82.5, 0.0],
    "pnl_amount": [0, 0, 1405, 2810, 0, 0, -45],
}

df = pd.DataFrame(data)

# Keep only the required columns
df["stock_name"] = "stock"
df["datetime"] = pd.to_datetime(df["datetime"])

# df = df[["datetime","stock_name", "side", "quantity", "price"]].sort_values(by="datetime")


# # Calculate cumulative sum using a loop
# holding_amount = 0
# for i in range(len(df)):
#     if df.loc[i,"quantity"] >= df.loc[i,"holding_quantity"]:
#         holding_amount += df.loc[i,"quantity"] * df.loc[i,"price"]

#     df.loc[i, 'holding_amount'] = holding_amount

# df["avg_price"] = 0

# # # df['old_holding_quantity'] = df.groupby(['stock_name'])['holding_quantity'].shift(1).fillna(0)
# # # Calculate the P&L amount
# # df["pnl_amount"] = (df["avg_price"] - df["price"]) * df["quantity"]

# df.replace([np.inf, -np.inf], 0, inplace=True)
df

,datetime,side,quantity,price,amount,avg_price,holding_quantity,holding_amount,pnl_amount,stock_name
0,2024-07-09 12:44:04,BUY,25,89.0,2225.0,89.0,25,2225.0,0,stock
1,2024-07-09 12:44:04,BUY,125,89.0,11125.0,89.0,150,13350.0,0,stock
2,2024-07-09 13:17:26,SELL,50,117.1,5855.0,89.0,100,8900.0,1405,stock
3,2024-07-09 13:17:26,SELL,100,117.1,11710.0,0.0,0,0.0,2810,stock
4,2024-07-11 11:05:32,SELL,25,4.3,107.5,4.3,-25,-107.5,0,stock
5,2024-07-11 14:09:06,BUY,10,2.5,25.0,5.5,-15,-82.5,0,stock
6,2024-07-11 14:09:06,BUY,15,2.5,37.5,0.0,0,0.0,-45,stock


In [60]:
import pandas as pd

# Sample data
data = {
    "datetime": [
        "09-07-2024 12:44",
        "09-07-2024 12:44",
        "09-07-2024 13:17",
        "09-07-2024 13:17",
        "11-07-2024 11:05",
        "11-07-2024 11:30",
        "11-07-2024 14:09",
        "11-07-2024 14:19",
    ],
    "stock_name": ["stock"] * 8,
    "side": ["BUY", "BUY", "SELL", "SELL", "SELL", "SELL", "BUY", "BUY"],
    "quantity": [5, 10, 5, 10, 20, 5, 10, 15],
    "price": [100, 91, 95, 100, 100, 120, 50, 40],
}

df = pd.DataFrame(data)
# Adjust the quantity for SELL transactions
df["quantity"] = df["quantity"] * df["side"].apply(lambda x: -1 if x == "SELL" else 1)

# Calculate the amount
df["amount"] = df["quantity"] * df["price"]

# Calculate the cumulative sum of quantity
df["holding_quantity"] = df["quantity"].cumsum()

# Initialize new columns
df["holding_amount"] = 0
df["avg_price"] = 0
df["pnl"] = 0

# Calculate the columns based on the given logic
for i in range(len(df)):
    if i == 0:
        df.at[i, "holding_amount"] = df.at[i, "amount"]
    else:
        if df.at[i, "side"] == "BUY":
            df.at[i, "holding_amount"] = df.at[i - 1, "holding_amount"] + df.at[i, "amount"]
        else:
            df.at[i, "holding_amount"] = df.at[i - 1, "holding_amount"] - df.at[i - 1, "avg_price"] * df.at[i, "quantity"]
            df.at[i, "pnl"] = (df.at[i, "price"] - df.at[i - 1, "avg_price"]) * df.at[i, "quantity"]

    if df.at[i, "holding_quantity"] != 0:
        df.at[i, "avg_price"] = df.at[i, "holding_amount"] / df.at[i, "holding_quantity"]
    else:
        df.at[i, "avg_price"] = 0

# Adjust 'avg_price' for any initial rows where it was 0 (to avoid divide by zero)
df["avg_price"].fillna(0, inplace=True)

df

,datetime,stock_name,side,quantity,price,amount,holding_quantity,holding_amount,avg_price,pnl
0,09-07-2024 12:44,stock,BUY,5,100,500,5,500,100.000000,0
1,09-07-2024 12:44,stock,BUY,10,91,910,15,1410,94.000000,0
2,09-07-2024 13:17,stock,SELL,-5,95,-475,10,1880,188.000000,-5
3,09-07-2024 13:17,stock,SELL,-10,100,-1000,0,3760,0.000000,880
4,11-07-2024 11:05,stock,SELL,-20,100,-2000,-20,3760,-188.000000,-2000
5,11-07-2024 11:30,stock,SELL,-5,120,-600,-25,2820,-112.800000,-1540
6,11-07-2024 14:09,stock,BUY,10,50,500,-15,3320,-221.333333,0
7,11-07-2024 14:19,stock,BUY,15,40,600,0,3920,0.000000,0
